# Example usage of lattedb

In [1]:
from typing import Dict, Any

import os
import re

from lattedb.correlator.models import Baryon2pt

In [2]:
%load_ext blackcellmagic

In [3]:
CWD = os.getcwd()

In [4]:
def parse_ensemble(short_tag: str) -> Dict[str, Any]:

    long_tag = {
        "a15m310L": "l2448f211b580m013m065m838",
        "a12m310": "l2464f211b600m0102m0509m635",
        "a09m310": "l3296f211b630m0074m037m440",
    }
    key = long_tag[short_tag]

    pattern = (
        "l"
        "(?P<nx>[0-9]{2})"
        "(?P<nt>[0-9]{2})"
        "f211"
        "b(?P<beta>[0-9]{3})"
        "m(?P<light__quark_mass>[0-9]+)"
        "m(?P<strange__quark_mass>[0-9]+)"
        "m(?P<charm__quark_mass>[0-9]+)"
    )
    type_map = {
        "nx": str,
        "nt": str,
        "beta": lambda beta: ".".join([beta[:-2], beta[-2:]]),
        "light__quark_mass": lambda mass: f"0.{mass}",
        "strange__quark_mass": lambda mass: f"0.{mass}",
        "charm__quark_mass": lambda mass: f"0.{mass}",
    }

    match = re.match(pattern, key)

    info = (
        {
            key: type_map[key](val)
            for key, val in match.groupdict().items()
            if key in type_map
        }
        if match
        else {}
    )

    ## from https://arxiv.org/pdf/1212.4768.pdf TAB IV

    charm_naik = {"5.80": "-0.3582", "6.00": "-0.2308", "6.30": "-0.1204"}
    info["light__naik"] = "0.0"
    info["strange__naik"] = "0.0"
    info["charm__naik"] = charm_naik[info["beta"]]

    u0 = {"5.80": "0.8553", "6.00": "0.86372", "6.30": "0.874164"}
    info["u0"] = u0[info["beta"]]

    return info


parse_ensemble("a09m310")

{'nx': '32',
 'nt': '96',
 'beta': '6.30',
 'light__quark_mass': '0.0074',
 'strange__quark_mass': '0.037',
 'charm__quark_mass': '0.440',
 'light__naik': '0.0',
 'strange__naik': '0.0',
 'charm__naik': '-0.1204',
 'u0': '0.874164'}

In [5]:
def parse_propagator(key) -> Dict[str, Any]:
    pattern = (
        "(?P<config>[0-9]+)"
        "/"
        "prop_"
        "(?P<short_tag>a[0-9]+m[0-9]+)"
        "_"
        "(?P<stream>[a-z]{1})"
        "_[0-9]+_"
        "gf(?P<flowtime>[0-9\.]+)"
        "_"
        "w(?P<radius>[0-9\.]+)"
        "_"
        "n(?P<step>[0-9]+)"
        "_"
        "M5(?P<m5>[0-9\.]+)"
        "_"
        "L5(?P<l5>[0-9]+)"
        "_"
        "a(?P<alpha5>[0-9\.]+)"
        "_"
        "mq(?P<MobiusDWF__quark_mass>[0-9\.]+)"
        "_"
        "x(?P<origin_x>[0-9]+)"
        "y(?P<origin_y>[0-9]+)"
        "z(?P<origin_z>[0-9]+)"
        "t(?P<origin_t>[0-9]+)"
    )
    match = re.match(pattern, key)
    
    info = match.groupdict() if match else {}
    
    info["a_fm"] = info["short_tag"].replace("a", "0.").split("m")[0]
    info["mpi"] = info["short_tag"].replace("a", "0.").split("m")[1]
    
    ## https://c51.lbl.gov/wiki/mdwf_hisq existing ensembles
    c5 = {
        "a09m310": "0.25",
        "a12m310": "0.25", 
        "a15m310L": "0.5",    
    }

    b5 = {
        "a09m310": "1.25",
        "a12m310": "1.25", 
        "a15m310L": "1.5",    
    }

    info["c5"] = c5[info["short_tag"]]
    info["b5"] = c5[info["short_tag"]]
    
    info.update(parse_ensemble(info["short_tag"]))
    
    return info


key = "300/prop_a09m310_e_300_gf1.0_w3.5_n45_M51.1_L56_a1.5_mq0.00951_x3y3z19t62.lime"

parse_propagator(key)

{'config': '300',
 'short_tag': 'a09m310',
 'stream': 'e',
 'flowtime': '1.0',
 'radius': '3.5',
 'step': '45',
 'm5': '1.1',
 'l5': '6',
 'alpha5': '1.5',
 'MobiusDWF__quark_mass': '0.00951',
 'origin_x': '3',
 'origin_y': '3',
 'origin_z': '19',
 'origin_t': '62',
 'a_fm': '0.09',
 'mpi': '310',
 'c5': '0.25',
 'b5': '0.25',
 'nx': '32',
 'nt': '96',
 'beta': '6.30',
 'light__quark_mass': '0.0074',
 'strange__quark_mass': '0.037',
 'charm__quark_mass': '0.440',
 'light__naik': '0.0',
 'strange__naik': '0.0',
 'charm__naik': '-0.1204',
 'u0': '0.874164'}

In [6]:
# large needs to be filled dynamically

large_int = 1
large_float = 1.1
large = 1
large_string = "1"

global_pars = {
    # Additional Gauge Config Smearing
    "flowstep": 40,
    "flowtime": large,
    # Gauge Config
    "config": large_int,
    "l_fm": large_float,
    "nt": large_int,
    "nx": large_int,
    "ny": large_int,
    "nz": large_int,
    "mpi": 310,
    "short_tag": large_string,
    "stream": large_string,
    # Gauge action
    "beta": large_float,
    "a_fm": large_float,
    "u0": large,
    # OneToAll
    "origin_x": large_int,
    "origin_y": large_int,
    "origin_z": large_int,
    "origin_t": large_int,
    # MobiusDW
    "b5": large_float, ##NEED
    "c5": large_float, ##NEED
    "l5": large_int,
    "m5": large_float,
    # Hisq
    "naik": large,
    # Hadron Gaussian Smear
    "radius": large_int,
    "step": large_float,
    # Interpolator: Hadron
    "strangeness": 0,
    "description": "nucleon interpolation operator [hep-lat/0508018]",
    "irrep": "g1",
    "embedding": 1,
    "parity": [1,-1],
    "spin_x2": 1,
    "spin_z_x2": [1,-1],
    "isospin_x2": 1,
    "isospin_z_x2": [1,-1],
    "momentum": 0,
}

In [7]:
special_pars = {
    ## Specializations ##
    # Valence
    # hack to resolve same instanziation fix
    "propagator0.fermionaction.quark_mass": large_float,
    "propagator1.fermionaction.quark_mass": large_float,
    "propagator2.fermionaction.quark_mass": large_float,
    "propagator0.fermionaction.quark_tag": "light",
    "propagator1.fermionaction.quark_tag": "light",
    "propagator2.fermionaction.quark_tag": "light",
    # Sea
    "propagator0.gaugeconfig.light.quark_mass": large_float,
    "propagator1.gaugeconfig.light.quark_mass": large_float,
    "propagator2.gaugeconfig.light.quark_mass": large_float,
    "propagator0.gaugeconfig.strange.quark_mass": large_float,
    "propagator1.gaugeconfig.strange.quark_mass": large_float,
    "propagator2.gaugeconfig.strange.quark_mass": large_float,
    "propagator0.gaugeconfig.charm.quark_mass": large_float,
    "propagator1.gaugeconfig.charm.quark_mass": large_float,
    "propagator2.gaugeconfig.charm.quark_mass": large_float,
    "propagator0.gaugeconfig.light.quark_tag": "light",
    "propagator1.gaugeconfig.light.quark_tag": "light",
    "propagator2.gaugeconfig.light.quark_tag": "light",
    "propagator0.gaugeconfig.strange.quark_tag": "strange",
    "propagator1.gaugeconfig.strange.quark_tag": "strange",
    "propagator2.gaugeconfig.strange.quark_tag": "strange",
    "propagator0.gaugeconfig.charm.quark_tag": "charm",
    "propagator1.gaugeconfig.charm.quark_tag": "charm",
    "propagator2.gaugeconfig.charm.quark_tag": "charm",
}


In [8]:
tree = {
    "propagator0": "OneToAll",
    "propagator0.fermionaction": "MobiusDW",
    "propagator0.gaugeconfig": "Nf211",
    "propagator0.gaugeconfig.light": "Hisq",
    "propagator0.gaugeconfig.strange": "Hisq",
    "propagator0.gaugeconfig.charm": "Hisq",
    "propagator0.gaugeconfig.gaugeaction": "LuescherWeisz",
    "propagator0.gaugeconfig.gaugesmear": "WilsonFlow",
    "propagator1": "OneToAll",
    "propagator1.fermionaction": "MobiusDW",
    "propagator1.gaugeconfig": "Nf211",
    "propagator1.gaugeconfig.light": "Hisq",
    "propagator1.gaugeconfig.strange": "Hisq",
    "propagator1.gaugeconfig.charm": "Hisq",
    "propagator1.gaugeconfig.gaugeaction": "LuescherWeisz",
    "propagator1.gaugeconfig.gaugesmear": "WilsonFlow",
    "propagator2": "OneToAll",
    "propagator2.fermionaction": "MobiusDW",
    "propagator2.gaugeconfig": "Nf211",
    "propagator2.gaugeconfig.light": "Hisq",
    "propagator2.gaugeconfig.strange": "Hisq",
    "propagator2.gaugeconfig.charm": "Hisq",
    "propagator2.gaugeconfig.gaugeaction": "LuescherWeisz",
    "propagator2.gaugeconfig.gaugesmear": "WilsonFlow",
    "sink": "Hadron",
    "sink.interpolatorsmear": "Gaussian",
    "source": "Hadron4D",
    "source.interpolatorsmear": "Gaussian",
}

In [9]:
parameters = {**global_pars, **special_pars}

In [10]:
Baryon2pt.get_or_create_from_parameters(parameters=parameters, tree=tree)

Column tag is used by the following tables ['Baryon2pt', 'Baryon2pt.OneToAll', 'Baryon2pt.OneToAll.Nf211', 'Baryon2pt.OneToAll.Nf211.LuescherWeisz', 'Baryon2pt.OneToAll.Nf211.WilsonFlow', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.MobiusDW', 'Baryon2pt.OneToAll', 'Baryon2pt.OneToAll.Nf211', 'Baryon2pt.OneToAll.Nf211.LuescherWeisz', 'Baryon2pt.OneToAll.Nf211.WilsonFlow', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.MobiusDW', 'Baryon2pt.OneToAll', 'Baryon2pt.OneToAll.Nf211', 'Baryon2pt.OneToAll.Nf211.LuescherWeisz', 'Baryon2pt.OneToAll.Nf211.WilsonFlow', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.Nf211.Hisq', 'Baryon2pt.OneToAll.MobiusDW', 'Baryon2pt.Hadron4D', 'Baryon2pt.Hadron4D.Gaussian', 'Baryon2pt.Hadron', 'Baryon2pt.Hadron.Gaussian']
Column misc is used by the following tables ['Baryon2pt', 

|----Created Nf211[GaugeConfig](short_tag=1, stream=1, config=1, nx=1, ny=1, nz=1, nt=1, l_fm=1.100000000, mpi=310)
|----Preparing creation of <class 'lattedb.fermionaction.models.MobiusDW'>
|----Fetched MobiusDW[FermionAction](quark_mass=1.100000, quark_tag=light, l5=1, m5=1.100000, b5=1.100000, c5=1.100000) from db
|--Created OneToAll[Propagator](origin_x=1, origin_y=1, origin_z=1, origin_t=1)
|--Preparing creation of <class 'lattedb.propagator.models.OneToAll'>
|----Preparing creation of <class 'lattedb.gaugeconfig.models.Nf211'>
|------Preparing creation of <class 'lattedb.gaugeaction.models.LuescherWeisz'>
|------Fetched LuescherWeisz[GaugeAction](beta=1.100000, a_fm=1.100000, u0=1.000000) from db
|------Preparing creation of <class 'lattedb.gaugesmear.models.WilsonFlow'>
|------Fetched WilsonFlow[GaugeSmear](flowtime=1.000000, flowstep=40) from db
|------Preparing creation of <class 'lattedb.fermionaction.models.Hisq'>
|------Fetched Hisq[FermionAction](quark_mass=1.100000, quark

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'